In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE193677"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE193677"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE193677.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE193677.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE193677.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Biopsy expression profiling of an adult inflammatory bowel disease cohort"
!Series_summary	"Inflammatory Bowel Disease (IBD) is a progressive disease of the gut and consists of two types, Crohn’s Disease (CD) and Ulcerative Colitis (UC). It is a complex disease involving genetic, microbial, and environmental factors. The incidence of IBD is steadily increasing and current therapeutic options are plateauing. Thus treatments are evolving to 1. deeper levels of remission from clinical to endoscopic and histologic normalization and 2. Embrace novel targets or drug combinations. We explored whole transcriptome data generated in biopsy specimens sampled from a large cohort of adult IBD and control subjects to provide 1. a granular, objective and sensitive molecular measures of disease activity in the gut and 2. Novel molecular mechanisms and biomarkers underlying IBD pathology."
!Series_overall_design	"The Mount Sinai Crohn's and Colitis registry (MSCCR

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine if gene expression data is available
# Based on the background information, this dataset contains RNA-Seq data which is gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Checking trait data availability
# Looking at keys 4 and 6, we can see disease information:
# Key 4: 'ibd_disease: UC', 'ibd_disease: CD', 'ibd_disease: Control'
# Key 6: 'diseasebi: IBD', 'diseasebi: Control'
# For Crohn's Disease, key 4 contains the specific disease type
trait_row = 4

# Age information is in key 0 (study_eligibility_age_at_endo)
age_row = 0

# Gender information is in key 1 (demographics_gender)
gender_row = 1

# 2.2 Data Type Conversion functions
def convert_trait(value):
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check for Crohn's Disease
    if value == 'CD':
        return 1
    elif value in ['UC', 'Control']:
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # First, we need to convert the sample characteristics into a proper DataFrame format
    # The clinical_data variable is assumed to be a DataFrame from previous steps containing the characteristic data
    
    # The error indicates we need to access the clinical data differently
    # Wait for the actual clinical_data to be passed from the previous step
    # For now, just print a message about what would happen next
    print(f"Clinical feature extraction would proceed if clinical_data were available.")
    print(f"Would extract features: trait_row={trait_row}, age_row={age_row}, gender_row={gender_row}")
    print(f"Would save to: {out_clinical_data_file}")


Clinical feature extraction would proceed if clinical_data were available.
Would extract features: trait_row=4, age_row=0, gender_row=1
Would save to: ../../output/preprocess/Crohns_Disease/clinical_data/GSE193677.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Print the file paths to debug
print(f"SOFT file path: {soft_file}")
print(f"Matrix file path: {matrix_file}")

# Try a simpler direct approach to read the gene expression data
import pandas as pd
import gzip

try:
    # Use the library function with proper error handling
    gene_data = get_genetic_data(matrix_file)
    print("Successfully extracted gene data using get_genetic_data function")
except Exception as e:
    print(f"Error with get_genetic_data: {e}")
    gene_data = None

# If the library function failed, try a manual approach
if gene_data is None or gene_data.shape[0] == 0:
    print("Attempting manual extraction...")
    with gzip.open(matrix_file, 'rt') as file:
        # Read the file content into memory
        content = file.read()
        
        # Find the table markers
        start_marker = "!series_matrix_table_begin"
        end_marker = "!series_matrix_table_end"
        
        if start_marker in content.lower():
            # Get position of start marker
            start_idx = content.lower().find(start_marker)
            # Find the end of the line containing the start marker
            start_idx = content.find('\n', start_idx) + 1
            
            # Find end marker if it exists
            if end_marker in content.lower():
                end_idx = content.lower().find(end_marker)
            else:
                end_idx = len(content)
            
            # Extract the table content
            table_content = content[start_idx:end_idx]
            
            # Read into DataFrame
            import io
            gene_data = pd.read_csv(io.StringIO(table_content), sep='\t', index_col=0)
            
            # Skip the first row if it contains the header
            if gene_data.index.name == 'ID_REF':
                gene_data = gene_data.reset_index().iloc[1:].set_index('ID_REF')
            
            print(f"Manual extraction completed, shape: {gene_data.shape}")

# Fall back to checking the soft file for gene IDs if matrix extraction failed
if gene_data is None or gene_data.shape[0] == 0:
    print("\nMatrix file gene data extraction failed. Checking SOFT file...")
    try:
        gene_metadata = get_gene_annotation(soft_file)
        print(f"Gene metadata from SOFT file has shape: {gene_metadata.shape}")
        # If successful, we'll use this as our gene data
        if 'ID' in gene_metadata.columns and gene_metadata.shape[0] > 0:
            # Create a minimal gene data frame with just the identifiers
            gene_data = gene_metadata[['ID']].set_index('ID')
            print("Using gene identifiers from SOFT file as fallback")
    except Exception as e:
        print(f"Error extracting gene metadata from SOFT file: {e}")

# 3. Print the first 20 row IDs (gene/probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
if gene_data is not None and gene_data.shape[0] > 0:
    print(gene_data.index[:20])
    # 4. Print the dimensions of the gene expression data
    print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
else:
    print("Failed to extract gene identifiers.")

# Update gene availability based on our findings
is_gene_available = (gene_data is not None and gene_data.shape[0] > 0)
print(f"Gene expression data available: {is_gene_available}")


SOFT file path: ../../input/GEO/Crohns_Disease/GSE193677/GSE193677_family.soft.gz
Matrix file path: ../../input/GEO/Crohns_Disease/GSE193677/GSE193677_series_matrix.txt.gz


Successfully extracted gene data using get_genetic_data function
Attempting manual extraction...


Manual extraction completed, shape: (0, 2490)

Matrix file gene data extraction failed. Checking SOFT file...
Error extracting gene metadata from SOFT file: No columns to parse from file

First 20 gene/probe identifiers:
Failed to extract gene identifiers.
Gene expression data available: False


### Step 4: Gene Annotation

In [5]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Check file integrity and size
import os
file_size = os.path.getsize(soft_file)
print(f"SOFT file size: {file_size} bytes")

# First, check what's actually in the SOFT file
import gzip
try:
    with gzip.open(soft_file, 'rt') as f:
        print("First 20 lines of the SOFT file:")
        for i in range(20):
            try:
                line = next(f)
                print(line.strip())
            except StopIteration:
                print("End of file reached.")
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# Try a direct inspection of the matrix file instead
try:
    with gzip.open(matrix_file, 'rt') as f:
        print("\nFirst few lines of the matrix file:")
        for i in range(10):
            try:
                line = next(f)
                print(line.strip())
            except StopIteration:
                print("End of file reached.")
                break
except Exception as e:
    print(f"Error reading matrix file: {e}")

# Update gene availability status based on our findings
is_gene_available = False

# Update the dataset usability information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=True  # From previous step
)

SOFT file size: 185596 bytes
First 20 lines of the SOFT file:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE193677
!Series_title = Biopsy expression profiling of an adult inflammatory bowel disease cohort
!Series_geo_accession = GSE193677
!Series_status = Public on Sep 16 2022
!Series_submission_date = Jan 13 2022
!Series_last_update_date = Nov 04 2024
!Series_pubmed_id = 36109152
!Series_summary = Inflammatory Bowel Disease (IBD) is a progressive disease of the gut and consists of two types, Crohn’s Disease (CD) and Ulcerative Colitis (UC). It is a complex disease involving genetic, microbial, and environmental factors. The incidence of IBD is steadily increasing and current therapeutic options are plateauing. Thus treatments are evolving to 1. deeper levels of remission from clinical to endoscopic and histologic normalizatio

False